#### <b>GAN 실습</b>

* 논문 제목: Generative Adversarial Networks <b>(NIPS 2014)</b>
* 가장 기본적인 GAN 모델을 학습해보는 실습을 진행합니다.
* 학습 데이터셋: <b>MNIST</b> (1 X 28 X 28)

#### <b>필요한 라이브러리 불러오기</b>

* 실습을 위한 PyTorch 라이브러리를 불러옵니다.

In [21]:

import torch
import torch.nn as nn

from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image
import os

os.makedirs('_saved_models', exist_ok=True)

#### <b>생성자(Generator) 및 판별자(Discriminator) 모델 정의</b>

In [22]:
latent_dim = 100


# 생성자(Generator) 클래스 정의
class Generator(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super(Generator, self).__init__()

        # First layer
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels, 2, kernel_size=3, stride=1, padding=1), nn.ReLU())
        self.conv2 = nn.Sequential(nn.Conv2d(2, 4, kernel_size=3, stride=1, padding=1), nn.ReLU())
        self.conv3 = nn.Sequential(nn.Conv2d(4, 8, kernel_size=3, stride=1, padding=1), nn.ReLU())
        self.conv4 = nn.Sequential(nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1), nn.ReLU())
        self.conv5 = nn.Sequential(nn.Conv2d(16, 8, kernel_size=3, stride=1, padding=1), nn.ReLU())
        self.conv6 = nn.Sequential(nn.Conv2d(8, 4, kernel_size=3, stride=1, padding=1), nn.ReLU())
        self.conv7 = nn.Sequential(nn.Conv2d(4, 2, kernel_size=3, stride=1, padding=1), nn.ReLU())
        self.conv8 = nn.Sequential(nn.Conv2d(2, 1, kernel_size=3, stride=1, padding=1), nn.ReLU())

    def forward(self, x):
        out1 = self.conv1(x)
        out2 = self.conv2(out1)
        out3 = self.conv3(out2)
        out4 = self.conv4(out3)
        out5 = self.conv5(out4) +out3
        out6 = self.conv6(out5) +out2
        out7 = self.conv7(out6) +out1
        out8 = self.conv8(out7)
        return out8

        

#### <b>학습 데이터셋 불러오기</b>

* 학습을 위해 MNIST 데이터셋을 불러옵니다.

In [23]:
transforms_train = transforms.Compose([
    transforms.Resize(28),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

train_dataset = datasets.MNIST(root="./dataset", train=True, download=True, transform=transforms_train)
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)



def mask(X,coords):
  x0,y0,x1,y1 = coords
  X[:,x0:x1,y0:y1] = 0
  return X


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


#### <b>모델 학습 및 샘플링</b>

* 학습을 위해 생성자와 판별자 모델을 초기화합니다.
* 적절한 하이퍼 파라미터를 설정합니다.

In [24]:
# 생성자(generator)와 판별자(discriminator) 초기화
generator = Generator()

generator.cuda()


load_pretrained_models = False
saved_epoch =3
if load_pretrained_models:
    generator.load_state_dict(torch.load('_saved_models/generator_' + str(saved_epoch) + '.pth'))
# 손실 함수(loss function)
MSELoss = torch.nn.L1Loss()
MSELoss.cuda()

# 학습률(learning rate) 설정
lr = 0.0002

# 생성자와 판별자를 위한 최적화 함수
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))

* 모델을 학습하면서 주기적으로 샘플링하여 결과를 확인할 수 있습니다.

In [ ]:
import time

n_epochs = 1000 # 학습의 횟수(epoch) 설정
sample_interval = 2000 # 몇 번의 배치(batch)마다 결과를 출력할 것인지 설정
start_time = time.time()

for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        real_imgs = imgs.cuda()

        masked_imgs = mask(real_imgs.clone(),(0,0,10,10))
        """ 생성자(generator)를 학습합니다. """
        optimizer_G.zero_grad()

        # 랜덤 노이즈(noise) 샘플링
        # 이미지 생성
        generated_imgs = generator(masked_imgs)

        # 생성자(generator)의 손실(loss) 값 계산
        g_loss = MSELoss(generated_imgs, real_imgs)

        # 생성자(generator) 업데이트
        g_loss.backward()
        optimizer_G.step()
        done = epoch * len(dataloader) + i
        torch.save(generator.state_dict(), '_saved_models/generator_' + str(epoch) + '.pth')
        if done % sample_interval == 0:
            # 생성된 이미지 중에서 25개만 선택하여 5 X 5 격자 이미지에 출력
            save_image(generated_imgs.data[:25], f"{done}generated.png", nrow=5, normalize=True)
            save_image(masked_imgs.data[:25], f"{done}mask.png", nrow=5, normalize=True)
            save_image(real_imgs.data[:25], f"{done}real.png", nrow=5, normalize=True)
    # 하나의 epoch이 끝날 때마다 로그(log) 출력
    print(f"[Epoch {epoch}/{n_epochs}] [G loss: {g_loss.item():.6f}] [Elapsed time: {time.time() - start_time:.2f}s]")

* 생성된 이미지 예시를 출력합니다.

In [ ]:
from IPython.display import Image

Image('92000.png')